In [1]:
%pip install --upgrade --quiet google-cloud-aiplatform requests

Note: you may need to restart the kernel to use updated packages.


In [32]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [44]:
import requests
import vertexai
from vertexai.generative_models import (
    Content,
    FunctionDeclaration,
    GenerationConfig,
    GenerativeModel,
    Part,
    Tool,
)

# Initialize Vertex AI
vertexai.init()


In [45]:
# Initialize Gemini model
model = GenerativeModel("gemini-1.5-flash-001",
                        system_instruction=["""You are a store support API assistant to help with online orders."""])

# Define the user's prompt in a Content object that we can reuse in model calls
user_prompt_content = Content(
    role="user",
    parts=[
        Part.from_text("What's the status of my order ID #12345?. If the order is not shipped, can you return it?"),
        #Part.from_text("I want to return my order with ID #12345?")
    ],
)

In [46]:
# Define the function declarations
get_order_status_func = FunctionDeclaration(
    name="get_order_status",
    description="Retrieve the current status of an order by its order ID.",
    parameters={
        "type": "object",
        "properties": {
            "order_id": {
                "type": "string",
                "description": "The unique identifier of the order."
            }
        },
        "required": ["order_id"]
    },
)

initiate_return_func = FunctionDeclaration(
    name="initiate_return",
    description="Initiate a return process for a given order ID.",
    parameters={
        "type": "object",
        "properties": {
            "order_id": {
                "type": "string",
                "description": "The unique identifier of the order to be returned."
            }
        },
        "required": ["order_id"]
    },
)

cancel_order_func = FunctionDeclaration(
    name="cancel_order",
    description="Cancel the order if the order exists by its order ID and if the order is not shipped yet.",
    parameters={
        "type": "object",
        "properties": {
            "order_id": {
                "type": "string",
                "description": "The unique identifier of the order."
            }
        },
        "required": ["order_id"]
    },
)
# Define the tools that include the above functions
support_tool = Tool(
    function_declarations=[get_order_status_func, initiate_return_func, cancel_order_func],
)


In [47]:
# Send the prompt and instruct the model to generate content using the Tool that you just created
response = model.generate_content(
    user_prompt_content,
    generation_config=GenerationConfig(temperature=0),
    tools=[support_tool],
)

In [48]:
# Define dummy functions for each operation
def get_order_status(args):
    order_id = args.get("order_id")
    # Simulated response
    return {
        "order_id": order_id,
        "shipping_status": "Not Shipped",
        "expected_delivery": "Next week",
        "status": "Active"
    }

def initiate_return(args):
    order_id = args.get("order_id")
    reason = args.get("reason", "No reason provided")
    # Simulated response
    return {
        "order_id": order_id,
        "return_status": "Return initiated successfully.",
        "return_label": "You will receive a return label shortly."
    }

def cancel_order(args):
    order_id = args.get("order_id")
    # Simulated response
    return {
        "order_id": order_id,
        "status": "Cancelled"
    }

# Map function names to their handlers
function_handlers = {
    "get_order_status": get_order_status,
    "initiate_return": initiate_return,
    "cancel_order": cancel_order,
}

In [49]:
print(response)

candidates {
  content {
    role: "model"
    parts {
      function_call {
        name: "get_order_status"
        args {
          fields {
            key: "order_id"
            value {
              string_value: "12345"
            }
          }
        }
      }
    }
  }
  finish_reason: STOP
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
    probability: NEGLIGIBLE
    probability_score: 0.111328125
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.0693359375
  }
  safety_ratings {
    category: HARM_CATEGORY_DANGEROUS_CONTENT
    probability: NEGLIGIBLE
    probability_score: 0.341796875
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.182617188
  }
  safety_ratings {
    category: HARM_CATEGORY_HARASSMENT
    probability: NEGLIGIBLE
    probability_score: 0.163085938
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.078125
  }
  safety_ratings {
    category: HARM_CATEGORY_SEXUALLY_EXPLICIT
    probability: NEGLIGIBLE
    proba

In [50]:
# Iterate over all the function calls in the model's response
for function_call in response.candidates[0].function_calls:
    print(function_call)
    function_name = function_call.name
    args = {key: value for key, value in function_call.args.items()}
    
    if function_name in function_handlers:
        # Call the appropriate function with the extracted arguments
        api_response = function_handlers[function_name](args)

        # Return the dummy API response to Gemini so it can generate a model response or request another function call
        response = model.generate_content(
            [
                user_prompt_content,  # User prompt
                response.candidates[0].content,  # Function call response
                Content(
                    parts=[
                        Part.from_function_response(
                            name=function_call.name,
                            response={"content": api_response},  # Return the function response to Gemini
                        ),
                    ],
                ),
            ],
            tools=[support_tool],
        )

        # Get the model response and print it
        print(response.text)

name: "get_order_status"
args {
  fields {
    key: "order_id"
    value {
      string_value: "12345"
    }
  }
}

Your order #12345 is currently active and not shipped yet, expected delivery is next week. Since the order is not shipped, you can return it. Do you want to initiate the return process? 



In [ ]:
# Send the prompt and instruct the model to generate content using the Tool that you just created
response = model.generate_content(
    user_prompt_content,
    generation_config=GenerationConfig(temperature=0),
    tools=[support_tool],
)